In [97]:
import pandas as pd
import numpy as np
import re

In [98]:
df = pd.read_csv('airbnb_greece.csv')
df.head()

,price,location,rooms,rating,detail_ratings,place_offers
0,€ 117,"Agios Pavlos, Greece",3 guests · · 1 bedroom · · 1 bed · · 1 bath,5.0 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['Bathtub', 'Hair dryer', 'Cleaning products',..."
1,€ 100,"Thessaloniki, Greece",5 guests · · 2 bedrooms · · 3 beds · · 1 bath,4.97 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['City skyline view', 'Hair dryer', 'Free wash..."
2,NaN,"Thessaloniki, Greece",4 guests · · 1 bedroom · · 2 beds · · 1 bath,5.0 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['Hair dryer', 'Shampoo', 'Hot water', 'Shower..."
3,€ 150,"Thessaloniki, Greece",4 guests · · 2 bedrooms · · 2 beds · · 2.5 ...,4.94 ·,"['Cleanliness4.9', 'Accuracy4.9', 'Communicati...","['Mountain view', 'City skyline view', 'Sea vi..."
4,€ 203,"Thessaloniki, Greece",4 guests · · 1 bedroom · · 2 beds · · 1 bath,NaN,[],"['Hair dryer', 'LUX shampoo', 'LUX body soap',..."


In [99]:
df.shape

(108, 6)

In [100]:
df.columns

Index(['price', 'location', 'rooms', 'rating', 'detail_ratings',
       'place_offers'],
      dtype='object')

In [101]:
df.isna().sum()

price             24
location           0
rooms              0
rating            14
detail_ratings     0
place_offers       0
dtype: int64

In [102]:
df.rename(columns={'price':'price(€)'},inplace=True)

In [103]:
df['price(€)'] = df['price(€)'].apply(lambda x:x.split('€')[1] if type(x) is str else x)

In [104]:
df['price(€)'] = df['price(€)'].astype('float')

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price(€)        84 non-null     float64
 1   location        108 non-null    object 
 2   rooms           108 non-null    object 
 3   rating          94 non-null     object 
 4   detail_ratings  108 non-null    object 
 5   place_offers    108 non-null    object 
dtypes: float64(1), object(5)
memory usage: 5.2+ KB


In [106]:
df['location'] = df['location'].apply(lambda x: x.split(',')[0])

In [107]:
df['rooms']

0         3 guests ·  · 1 bedroom ·  · 1 bed ·  · 1 bath
1       5 guests ·  · 2 bedrooms ·  · 3 beds ·  · 1 bath
2        4 guests ·  · 1 bedroom ·  · 2 beds ·  · 1 bath
3      4 guests ·  · 2 bedrooms ·  · 2 beds ·  · 2.5 ...
4        4 guests ·  · 1 bedroom ·  · 2 beds ·  · 1 bath
                             ...                        
103       4 guests ·  · 1 bedroom ·  · 1 bed ·  · 1 bath
104       2 guests ·  · 1 bedroom ·  · 1 bed ·  · 1 bath
105    5 guests ·  · 2 bedrooms ·  · 3 beds ·  · 2 baths
106     4 guests ·  · 2 bedrooms ·  · 2 beds ·  · 1 bath
107      4 guests ·  · 1 bedroom ·  · 3 beds ·  · 1 bath
Name: rooms, Length: 108, dtype: object

In [108]:
df[['Guests','Bedroom','Bed','Bath']] = df['rooms'].str.split('·  ·', expand=True)

In [109]:
cols = ['Guests','Bedroom','Bed','Bath']
for col in cols:
    df[col] = df[col].apply(lambda x: x.split()[0])

In [110]:
df.drop(columns='rooms',inplace=True)

In [111]:
def apply_regex(record):
    res = re.sub(r'[\"]|[\']|[\[]|[\]]|[0]', ' ', record).strip()
    res = re.sub(r'\s+','',res)
    res = res.split(',')
    res_list = []
    if len(res)<1:
        # res_list =  6*[np.nan]
        res_list = 6*['']
    for x in res:
        var = re.findall(r'\d+',x)
        var = '.'.join(var)
        res_list.append(var)
    
    ret_str = ','.join(res_list)
    return ret_str

In [112]:
df[['Cleanliness','Accuracy','Communication','Location','Checkin','Value']] = df['detail_ratings'].apply(apply_regex).str.split(',', expand=True)
df

,price(€),location,rating,detail_ratings,place_offers,Guests,Bedroom,Bed,Bath,Cleanliness,Accuracy,Communication,Location,Checkin,Value
0,117.0,Agios Pavlos,5.0 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['Bathtub', 'Hair dryer', 'Cleaning products',...",3,1,1,1,5,5,5,5,5,5
1,100.0,Thessaloniki,4.97 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['City skyline view', 'Hair dryer', 'Free wash...",5,2,3,1,5,5,4.9,4.9,5,5
2,NaN,Thessaloniki,5.0 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['Hair dryer', 'Shampoo', 'Hot water', 'Shower...",4,1,2,1,5,5,5,4.9,5,5
3,150.0,Thessaloniki,4.94 ·,"['Cleanliness4.9', 'Accuracy4.9', 'Communicati...","['Mountain view', 'City skyline view', 'Sea vi...",4,2,2,2.5,4.9,4.9,5,4.9,5,4.8
4,203.0,Thessaloniki,NaN,[],"['Hair dryer', 'LUX shampoo', 'LUX body soap',...",4,1,2,1,,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,30.0,Thessaloniki,4.93 ·,[],"['Hair dryer', 'Shampoo', 'Hot water', 'Shower...",4,1,1,1,,None,None,None,None,None
104,37.0,Thessaloniki,4.71 ·,"['Cleanliness4.8', 'Accuracy4.8', 'Communicati...","['Hair dryer', 'Shampoo', 'Hot water', 'Shower...",2,1,1,1,4.8,4.8,5,4.9,5,4.7
105,79.0,Thessaloniki,4.92 ·,"['Cleanliness5.0', 'Accuracy5.0', 'Communicati...","['Hair dryer', 'Shampoo', 'Hot water', 'Washer...",5,2,3,2,5,5,5,5,5,4.9
106,143.0,Thessaloniki,4.93 ·,[],"['Hair dryer', 'Shampoo', 'Hot water', 'Free w...",4,2,2,1,,None,None,None,None,None


In [113]:
df.drop(columns='detail_ratings',inplace=True)

In [114]:
df.head()

,price(€),location,rating,place_offers,Guests,Bedroom,Bed,Bath,Cleanliness,Accuracy,Communication,Location,Checkin,Value
0,117.0,Agios Pavlos,5.0 ·,"['Bathtub', 'Hair dryer', 'Cleaning products',...",3,1,1,1,5,5,5,5,5,5
1,100.0,Thessaloniki,4.97 ·,"['City skyline view', 'Hair dryer', 'Free wash...",5,2,3,1,5,5,4.9,4.9,5,5
2,NaN,Thessaloniki,5.0 ·,"['Hair dryer', 'Shampoo', 'Hot water', 'Shower...",4,1,2,1,5,5,5,4.9,5,5
3,150.0,Thessaloniki,4.94 ·,"['Mountain view', 'City skyline view', 'Sea vi...",4,2,2,2.5,4.9,4.9,5,4.9,5,4.8
4,203.0,Thessaloniki,NaN,"['Hair dryer', 'LUX shampoo', 'LUX body soap',...",4,1,2,1,,None,None,None,None,None


In [115]:
df['rating'] = df['rating'].apply(lambda x: x.split('·')[0] if type(x) is str else x)

In [116]:
df.columns

Index(['price(€)', 'location', 'rating', 'place_offers', 'Guests', 'Bedroom',
       'Bed', 'Bath', 'Cleanliness', 'Accuracy', 'Communication', 'Location',
       'Checkin', 'Value'],
      dtype='object')

In [117]:
df['Bedroom'] = df['Bedroom'].replace({'Studio':1})

In [118]:
df['Cleanliness'] = df['Cleanliness'].replace({'':np.nan})

In [119]:
numeric_cols = ['rating','Guests', 'Bedroom',
       'Bed', 'Bath', 'Cleanliness', 'Accuracy', 'Communication', 'Location',
       'Checkin', 'Value']

for col in numeric_cols:
    df[col] = df[col].astype('float')

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price(€)       84 non-null     float64
 1   location       108 non-null    object 
 2   rating         94 non-null     float64
 3   place_offers   108 non-null    object 
 4   Guests         108 non-null    float64
 5   Bedroom        108 non-null    float64
 6   Bed            108 non-null    float64
 7   Bath           108 non-null    float64
 8   Cleanliness    77 non-null     float64
 9   Accuracy       77 non-null     float64
 10  Communication  77 non-null     float64
 11  Location       77 non-null     float64
 12  Checkin        77 non-null     float64
 13  Value          77 non-null     float64
dtypes: float64(12), object(2)
memory usage: 11.9+ KB


In [121]:
df.head()

,price(€),location,rating,place_offers,Guests,Bedroom,Bed,Bath,Cleanliness,Accuracy,Communication,Location,Checkin,Value
0,117.0,Agios Pavlos,5.00,"['Bathtub', 'Hair dryer', 'Cleaning products',...",3.0,1.0,1.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0
1,100.0,Thessaloniki,4.97,"['City skyline view', 'Hair dryer', 'Free wash...",5.0,2.0,3.0,1.0,5.0,5.0,4.9,4.9,5.0,5.0
2,NaN,Thessaloniki,5.00,"['Hair dryer', 'Shampoo', 'Hot water', 'Shower...",4.0,1.0,2.0,1.0,5.0,5.0,5.0,4.9,5.0,5.0
3,150.0,Thessaloniki,4.94,"['Mountain view', 'City skyline view', 'Sea vi...",4.0,2.0,2.0,2.5,4.9,4.9,5.0,4.9,5.0,4.8
4,203.0,Thessaloniki,NaN,"['Hair dryer', 'LUX shampoo', 'LUX body soap',...",4.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
